## IMPORT DEPENDENCIES

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import os
from torchvision.models import resnet18
from tqdm import tqdm
import math
import torch.autograd as autograd
import numpy as np
from torch.utils.data import Subset

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device



'cuda'

## DOWNLOAD and TRANSFORM CIFAR10

In [2]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=4)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=4)

# Reduce dataset size by 80%, keep 20% of the data
train_size = int(0.2 * len(train_dataset))
test_size = int(0.2 * len(test_dataset))

train_indices = np.random.choice(len(train_dataset), train_size, replace=False)
test_indices = np.random.choice(len(test_dataset), test_size, replace=False)

train_subset = Subset(train_dataset, train_indices)
test_subset = Subset(test_dataset, test_indices)

train_loader = torch.utils.data.DataLoader(train_subset, batch_size=256, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_subset, batch_size=256, shuffle=False, num_workers=4)


100%|██████████| 170498071/170498071 [00:03<00:00, 46378856.81it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## CUSTOM SUBNET CONVOLUTION LAYER

In [3]:
############## GIVE SUBNET BASED ON SCORE i.e. Take TOP K and Prune the rest######################
class GetSubnet(autograd.Function):
    @staticmethod
    def forward(ctx, scores, k):
        # Get the subnetwork by sorting the scores and using the top k%
        out = scores.clone()
        _, idx = scores.flatten().sort()
        j = int((1 - k) * scores.numel())

        # flat_out and out access the same memory.
        flat_out = out.flatten()
        flat_out[idx[:j]] = 0
        flat_out[idx[j:]] = 1

        return out

    @staticmethod
    def backward(ctx, g):
        # send the gradient g straight-through on the backward pass.
        return g, None


#################################### CONVULATION TYPE ################################################
# Not learning weights, finding subnet based on top k score values
class SubnetConv(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.scores = nn.Parameter(torch.Tensor(self.weight.size()))
        nn.init.kaiming_uniform_(self.scores, a=math.sqrt(5))

        self.use_subset = True

        self.score_mask = None

    def set_prune_rate(self, prune_rate):
        self.prune_rate = prune_rate

    def init_weight_with_score(self, prune_rate):
        self.weight.data = self.weight.data * GetSubnet.apply(self.clamped_scores, prune_rate).data
        self.use_subset = False

    @property
    def clamped_scores(self):
        return self.scores.abs()

    def get_subnet(self):
        return GetSubnet.apply(self.clamped_scores, self.prune_rate).detach()

    def forward(self, x):
        if self.use_subset:
            subnet = GetSubnet.apply(self.clamped_scores, self.prune_rate)
            w = self.weight * subnet
        else:
            w = self.weight

        x = F.conv2d(x, w, self.bias, self.stride, self.padding, self.dilation, self.groups)

        return x
    
    def count_nonzero_params(self):
        """Counts and returns the number of non-zero and total parameters."""
        total_params = self.weight.numel()  # Total number of parameters (before pruning)

        if self.prune_rate > 0:
            subnet = self.get_subnet()  # Get the mask (subnetwork)
            non_zero_params = (subnet != 0).sum().item()  # Count non-zero parameters after pruning
        else:
            non_zero_params = total_params  # If no pruning, all parameters are used

        return non_zero_params, total_params
    
    
class SubnetConv_row(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.scores = nn.Parameter(torch.Tensor(self.weight.size()[:3]))
        nn.init.kaiming_uniform_(self.scores, a=math.sqrt(5))

        self.use_subset = True
        
    def set_prune_rate(self, prune_rate):
        self.prune_rate = prune_rate

    def init_weight_with_score(self, prune_rate):
        self.weight.data = self.weight.data * GetSubnet.apply(self.clamped_scores, prune_rate).view(self.weight.size()[0], self.weight.size()[1], self.weight.size()[2], 1).data
        self.use_subset = False

    @property
    def clamped_scores(self):
        return self.scores.abs()

    def get_subnet(self):
        return GetSubnet.apply(self.clamped_scores, self.prune_rate).detach()

    def forward(self, x):
        if self.use_subset:
            subnet = GetSubnet.apply(self.clamped_scores, self.prune_rate)
            w = self.weight * subnet.view(self.weight.size()[0], self.weight.size()[1], self.weight.size()[2], 1)
        else:
            w = self.weight

        x = F.conv2d(
            x, w, self.bias, self.stride, self.padding, self.dilation, self.groups
        )
        return x
    
    def count_nonzero_params(self):
        """Counts and returns the number of non-zero and total parameters."""
        total_params = self.weight.numel()  # Total number of parameters (before pruning)

        if self.prune_rate > 0:
            subnet = self.get_subnet() 
            non_zero_params = (subnet != 0).sum().item()  # Count non-zero parameters after pruning
        else:
            non_zero_params = total_params  # If no pruning, all parameters are used

        return non_zero_params, total_params

class SubnetConv_kernel(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.scores = nn.Parameter(torch.Tensor(self.weight.size()[:2]))
        nn.init.kaiming_uniform_(self.scores, a=math.sqrt(5))

        self.use_subset = True

    def set_prune_rate(self, prune_rate):
        self.prune_rate = prune_rate

    def init_weight_with_score(self, prune_rate):
        self.weight.data = self.weight.data * GetSubnet.apply(self.clamped_scores, prune_rate).view(self.weight.size()[0], self.weight.size()[1], 1, 1).data
        self.use_subset = False
        
    @property
    def clamped_scores(self):
        return self.scores.abs()

    def get_subnet(self):
        return GetSubnet.apply(self.clamped_scores, self.prune_rate).detach()

    def forward(self, x):
        if self.use_subset:
            subnet = GetSubnet.apply(self.clamped_scores, self.prune_rate)
            w = self.weight * subnet.view(self.weight.size()[0], self.weight.size()[1], 1, 1)
        else:
            w = self.weight

        x = F.conv2d(
            x, w, self.bias, self.stride, self.padding, self.dilation, self.groups
        )
        return x
    def count_nonzero_params(self):
        """Counts and returns the number of non-zero and total parameters."""
        total_params = self.weight.numel()  # Total number of parameters (before pruning)

        if self.prune_rate > 0:
            subnet = self.get_subnet()  # Get the mask (subnetwork)
            non_zero_params = (subnet != 0).sum().item()  # Count non-zero parameters after pruning
        else:
            non_zero_params = total_params  # If no pruning, all parameters are used

        return non_zero_params, total_params
#################################### BATCH NORMALIZATION TYPE ################################################ 
class NonAffineBatchNorm(nn.BatchNorm2d):
    def __init__(self, dim):
        super(NonAffineBatchNorm, self).__init__(dim, affine=False)

## UTILITY FUNCTION FOR INITIALIZATION METHOD

In [4]:
def _calculate_fan_in_and_fan_out(tensor):
    dimensions = tensor.dim()
    if dimensions < 2:
        raise ValueError("Fan in and fan out cannot be computed for tensor with fewer than 2 dimensions")

    num_input_fmaps = tensor.size(1)
    num_output_fmaps = tensor.size(0)
    receptive_field_size = 1
    if tensor.dim() > 2:
        for s in tensor.shape[2:]:
            receptive_field_size *= s
    fan_in = num_input_fmaps * receptive_field_size
    fan_out = num_output_fmaps * receptive_field_size

    return fan_in, fan_out


def _calculate_correct_fan(tensor, mode):
    mode = mode.lower()
    valid_modes = ['fan_in', 'fan_out']
    if mode not in valid_modes:
        raise ValueError(f"Mode {mode} not supported, please use one of {valid_modes}")
    fan_in, fan_out = _calculate_fan_in_and_fan_out(tensor)
    return fan_in if mode == 'fan_in' else fan_out


def calculate_gain(nonlinearity, param=None):
    if nonlinearity == 'relu':
        return math.sqrt(2.0)
    elif nonlinearity == 'linear' or nonlinearity == 'conv2d':
        return 1.0
    elif nonlinearity == 'leaky_relu':
        if param is None:
            negative_slope = 0.01
        else:
            negative_slope = param
        return math.sqrt(2.0 / (1 + negative_slope ** 2))
    else:
        raise ValueError(f"Unsupported nonlinearity: {nonlinearity}")


def signed_constant(tensor, mode='fan_in', nonlinearity='relu', scale_fan=False, prune_rate=0.0):
    fan = _calculate_correct_fan(tensor, mode)
    if scale_fan:
        fan = fan * (1 - prune_rate)
    gain = calculate_gain(nonlinearity)
    std = gain / math.sqrt(fan)
    with torch.no_grad():
        tensor.data = tensor.sign() * std

    return tensor

## RESNET BLOCK

In [5]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, prune_rate=0.5, conv_layer=None):
        super(BasicBlock, self).__init__()
        assert conv_layer is not None, "conv_layer must be provided"
        
        self.conv_layer = conv_layer 

        self.conv1 = self.conv_layer(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.conv1.set_prune_rate(prune_rate)
        self.bn1 = NonAffineBatchNorm(planes)

        self.conv2 = self.conv_layer(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.conv2.set_prune_rate(prune_rate)
        self.bn2 = NonAffineBatchNorm(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                self.conv_layer(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                NonAffineBatchNorm(self.expansion * planes)
            )
            self.shortcut[0].set_prune_rate(prune_rate)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1, prune_rate=0.5, conv_layer=None):
        super(Bottleneck, self).__init__()
        assert conv_layer is not None, "conv_layer must be provided"

        self.conv_layer = conv_layer 

        self.conv1 = self.conv_layer(in_planes, planes, kernel_size=1, bias=False)
        self.conv1.set_prune_rate(prune_rate)
        self.bn1 = NonAffineBatchNorm(planes)

        self.conv2 = self.conv_layer(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.conv2.set_prune_rate(prune_rate)
        self.bn2 = NonAffineBatchNorm(planes)

        self.conv3 = self.conv_layer(planes, self.expansion * planes, kernel_size=1, bias=False)
        self.conv3.set_prune_rate(prune_rate)
        self.bn3 = NonAffineBatchNorm(self.expansion * planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                self.conv_layer(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                NonAffineBatchNorm(self.expansion * planes)
            )
            self.shortcut[0].set_prune_rate(prune_rate)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10, prune_rate=0.5, initialization_method='kaiming_constant', conv_layer=None):
        super(ResNet, self).__init__()
        assert conv_layer is not None, "conv_layer must be provided"

        self.in_planes = 64
        self.prune_rate = prune_rate
        self.conv_layer = conv_layer 

        self.conv1 = self.conv_layer(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.conv1.set_prune_rate(prune_rate)
        self.bn1 = NonAffineBatchNorm(64)

        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1, prune_rate=prune_rate)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2, prune_rate=prune_rate)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2, prune_rate=prune_rate)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2, prune_rate=prune_rate)

        self.linear = nn.Linear(512*block.expansion, num_classes)
        
        # Initialize Weight 
        if initialization_method == "kaiming_constant":
            print("======> Initializing Weight with Signed Kaiming Constant")
            self.kaiming_constant()            
        elif initialization_method == "kaiming_uniform":
            print("======> Initializing Weight with Kaiming Uniform")
            self.kaiming_uniform()
        elif initialization_method == "kaiming_normal":
            print("======> Initializing Weight with Kaiming Normal")
            self.kaiming_normal()
        elif initialization_method == "xavier_normal":
            print("======> Initializing Weight with Xavier Normal")
            self.xavier_normal()
        else:
            print("Choose a valid Initialization Method !!!!!!!!!")
            exit()

    def _make_layer(self, block, planes, num_blocks, stride, prune_rate):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride, prune_rate, conv_layer=self.conv_layer))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

    
    # Initialization methods
    def kaiming_constant(self):
        for m in self.modules():
            if isinstance(m, self.conv_layer):
                signed_constant(m.weight, mode='fan_in', nonlinearity='relu', scale_fan=False, prune_rate=self.prune_rate)
            elif isinstance(m, NonAffineBatchNorm):
                if m.weight is not None:
                    nn.init.constant_(m.weight, 1)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                signed_constant(m.weight, mode='fan_in', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def kaiming_uniform(self):
        for m in self.modules():
            if isinstance(m, self.conv_layer):
                nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
            elif isinstance(m, NonAffineBatchNorm):
                if m.weight is not None:
                    nn.init.constant_(m.weight, 1)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def kaiming_normal(self):
        for m in self.modules():
            if isinstance(m, self.conv_layer):
                fan_in = _calculate_correct_fan(m.weight, mode='fan_in')
                fan_in_scaled = fan_in * (1 - self.prune_rate)
                gain = calculate_gain('relu')
                std = gain / math.sqrt(fan_in_scaled)
                with torch.no_grad():
                    m.weight.normal_(0, std)
            elif isinstance(m, NonAffineBatchNorm):
                if m.weight is not None:
                    nn.init.constant_(m.weight, 1)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def xavier_normal(self):
        for m in self.modules():
            if isinstance(m, self.conv_layer):
                nn.init.xavier_normal_(m.weight)
            elif isinstance(m, NonAffineBatchNorm):
                if m.weight is not None:
                    nn.init.constant_(m.weight, 1)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
    
    def count_total_params(self):
        """Counts and prints the total number of non-zero and total parameters after pruning."""
        total_non_zero_params = 0
        total_params = 0

        for name, module in self.named_modules():
            if isinstance(module, self.conv_layer):
                non_zero_params, layer_total_params = module.count_nonzero_params()
                total_non_zero_params += non_zero_params
                total_params += layer_total_params

                print(f"{name}: {non_zero_params} out of {layer_total_params} parameters are non-zero")

        print(f"Total non-zero parameters after pruning: {total_non_zero_params}")
        print(f"Total parameters before pruning: {total_params}")


def ResNet18(prune_rate, initialization_method, conv_layer):
    return ResNet(BasicBlock, [2, 2, 2, 2], prune_rate=prune_rate, initialization_method=initialization_method, conv_layer=conv_layer)

In [6]:
def freeze_model_weights(model):
    for n, m in model.named_modules():
        if hasattr(m, "weight") and m.weight is not None:
            print(f"==> No gradient to {n}.weight")
            m.weight.requires_grad = False
            if m.weight.grad is not None:
                print(f"==> Setting gradient of {n}.weight to None")
                m.weight.grad = None

            if hasattr(m, "bias") and m.bias is not None:
                print(f"==> No gradient to {n}.bias")
                m.bias.requires_grad = False

                if m.bias.grad is not None:
                    print(f"==> Setting gradient of {n}.bias to None")
                    m.bias.grad = None
                    


def get_model(prune_rate, initialization_method, conv_layer):
    print(f"=======> Set Pruning Rate {prune_rate}")
    model = ResNet18(prune_rate, initialization_method, conv_layer) # remaining ratio
    print("=> Freezing model weights")
    freeze_model_weights(model)
    model.count_total_params()
    return model

## PGD, TRAIN, TEST FUNCTION

In [7]:
class LinfPGDAttack(object):
    def __init__(self, model):
        self.model = model
        self.epsilon = 8/256.
        self.alpha = 2/256.
        
    def perturb(self, x_natural, y, k):
        x = x_natural.detach()
        x = x + torch.zeros_like(x).uniform_(-self.epsilon, self.epsilon)
        for i in range(k):
            x.requires_grad_()
            with torch.enable_grad():
                logits = self.model(x)
                loss = F.cross_entropy(logits, y)
            grad = torch.autograd.grad(loss, [x])[0]
            x = x.detach() + self.alpha * torch.sign(grad.detach())
            x = torch.min(torch.max(x, x_natural - self.epsilon), x_natural + self.epsilon)
            x = torch.clamp(x, 0, 1)
        return x


def train(train_loader, epoch, adversary, criterion, optimizer):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in tqdm(enumerate(train_loader), total=len(train_loader), desc="Training Batches"):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        adv = adversary.perturb(inputs, targets, 7)
        adv_outputs = net(adv)
        loss = criterion(adv_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = adv_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        

    print('\nTotal adversarial train accuarcy:', 100. * correct / total)
    print('Total adversarial train loss:', train_loss)

best_robust_acc = 0
def test(test_loader, epoch, adversary, criterion, optimizer):
    global best_robust_acc
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    benign_loss = 0
    adv_loss = 0
    benign_correct = 0
    adv_correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in tqdm(enumerate(test_loader), total=len(test_loader), desc="Testing Batches"):
            inputs, targets = inputs.to(device), targets.to(device)
            total += targets.size(0)

            outputs = net(inputs)
            loss = criterion(outputs, targets)
            benign_loss += loss.item()

            _, predicted = outputs.max(1)
            benign_correct += predicted.eq(targets).sum().item()

            adv = adversary.perturb(inputs, targets, 20)
            adv_outputs = net(adv)
            loss = criterion(adv_outputs, targets)
            adv_loss += loss.item()

            _, predicted = adv_outputs.max(1)
            adv_correct += predicted.eq(targets).sum().item()

    print('\nTotal natural test accuarcy:', 100. * benign_correct / total)
    print('Total robust test Accuarcy:', 100. * adv_correct / total)
    print('Total natural test loss:', benign_loss)
    print('Total robust test loss:', adv_loss)
    if adv_correct > best_robust_acc:
        best_robust_acc = adv_correct  # Update best robust accuracy
        state = {
            'net': net.state_dict()
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, f'./checkpoint/{file_name}_best_{prune_rate}.pth')
        with open(f'{file_name}_accuracy_log_{prune_rate}.txt', 'a') as f:
            f.write(f'Epoch {epoch}: Natural Acc {100. * benign_correct / total}: Robust Accuracy = {100. * adv_correct / total}%\n')
        print('Model Saved!')


def adjust_learning_rate(learning_rate, optimizer, epoch):
    lr = learning_rate
    if epoch >= 100:
        lr /= 10
    if epoch >= 150:
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


## DRIVER CODE

In [8]:
initialization_method = "kaiming_constant" # ['kaiming_constant', 'kaiming_uniform', 'kaiming_normal', 'xavier_normal']
sparsity_pattern = SubnetConv_row # SubnetConv SubnetConv_row SubnetConv_kernel
file_name = f'pgd_adversarial_training_{initialization_method}'
learning_rate = 0.1
epochs = 100
prune_rates = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]

for prune_rate in prune_rates:
    best_robust_acc = 0
    net = get_model(prune_rate = prune_rate, initialization_method = initialization_method, conv_layer = sparsity_pattern)
    
    net = net.to(device)
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

    adversary = LinfPGDAttack(net)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0005)

    for epoch in range(0, epochs):
        adjust_learning_rate(learning_rate, optimizer, epoch)
        train(train_loader, epoch, adversary, criterion, optimizer)
        test(test_loader, epoch, adversary, criterion, optimizer)

=======> Set Pruning Rate 0.05
======> Initializing Weight with Signed Kaiming Constant
=> Freezing model weights
==> No gradient to conv1.weight
==> No gradient to layer1.0.conv1.weight
==> No gradient to layer1.0.conv2.weight
==> No gradient to layer1.1.conv1.weight
==> No gradient to layer1.1.conv2.weight
==> No gradient to layer2.0.conv1.weight
==> No gradient to layer2.0.conv2.weight
==> No gradient to layer2.0.shortcut.0.weight
==> No gradient to layer2.1.conv1.weight
==> No gradient to layer2.1.conv2.weight
==> No gradient to layer3.0.conv1.weight
==> No gradient to layer3.0.conv2.weight
==> No gradient to layer3.0.shortcut.0.weight
==> No gradient to layer3.1.conv1.weight
==> No gradient to layer3.1.conv2.weight
==> No gradient to layer4.0.conv1.weight
==> No gradient to layer4.0.conv2.weight
==> No gradient to layer4.0.shortcut.0.weight
==> No gradient to layer4.1.conv1.weight
==> No gradient to layer4.1.conv2.weight
==> No gradient to linear.weight
==> No gradient to linear.b

Training Batches: 100%|██████████| 40/40 [00:31<00:00,  1.27it/s]


Total adversarial train accuarcy: 11.63
Total adversarial train loss: 111.6369571685791

[ Test epoch: 0 ]



Testing Batches: 100%|██████████| 8/8 [00:14<00:00,  1.75s/it]


Total natural test accuarcy: 15.9
Total robust test Accuarcy: 10.4
Total natural test loss: 17.503816843032837
Total robust test loss: 18.746160745620728
Model Saved!

[ Train epoch: 1 ]


...


Total natural test accuarcy: 14.4
Total robust test Accuarcy: 4.65
Total natural test loss: 27.493825435638428
Total robust test loss: 36.88978338241577

[ Train epoch: 99 ]



Training Batches: 100%|██████████| 40/40 [00:25<00:00,  1.59it/s]


Total adversarial train accuarcy: 7.36
Total adversarial train loss: 101.75715851783752

[ Test epoch: 99 ]



Testing Batches: 100%|██████████| 8/8 [00:11<00:00,  1.41s/it]


Total natural test accuarcy: 14.65
Total robust test Accuarcy: 6.3
Total natural test loss: 45.56152009963989
Total robust test loss: 62.78669309616089


In [9]:
# import matplotlib.pyplot as plt
# import numpy as np

# # Define data for the plot
# remaining_ratio = [0.005, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]
# robust_acc = [32.51, 45.7, 47.28, 46.82, 45.72, 45.8, 45.75, 43.31, 36.56, 28.28, 22.29]
# natural_acc = [52.98, 74.42, 79.67, 79.96, 79.77, 79.34, 76.33, 72.57, 63.3, 43.95, 39.56]



# # Create the plot for ResNet18 with red line
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# # Plot Robust Accuracy
# ax1.plot(remaining_ratio, robust_acc, 'r-o', label='ResNet18')
# ax1.axhline(y=51.00, color='red', linestyle='--', linewidth=2)
# ax1.set_title("(a) Robust Acc @ CIFAR-10", fontsize=16, fontweight='bold')
# ax1.set_xlabel("Remaining Ratio", fontsize=14, fontweight='bold')
# ax1.set_ylabel("Robust Accuracy (%)", fontsize=14, fontweight='bold')
# ax1.set_xlim(-0.05, 1)
# ax1.set_ylim(20, 55)
# ax1.grid(True)
# ax1.legend(fontsize=12)
# ax1.set_yticks(np.arange(30, 51, 5))

# ax1.set_xticks(np.arange(0, 1, 0.2))
# ax1.tick_params(axis='both', which='major', labelsize=12)

# # Plot Natural Accuracy
# ax2.plot(remaining_ratio, natural_acc, 'r-o', label='ResNet18')
# ax2.axhline(y=83.56, color='red', linestyle='--', linewidth=2)
# ax2.set_title("(b) Natural Acc @ CIFAR-10", fontsize=16, fontweight='bold')
# ax2.set_xlabel("Remaining Ratio", fontsize=14, fontweight='bold')
# ax2.set_ylabel("Natural Accuracy (%)", fontsize=14, fontweight='bold')
# ax2.set_xlim(-0.05, 1)
# ax2.set_ylim(35, 90)
# ax2.grid(True)
# ax2.legend(fontsize=12)
# ax2.set_yticks(np.arange(50, 85, 5))
# ax2.set_xticks(np.arange(0, 1, 0.2))
# ax2.tick_params(axis='both', which='major', labelsize=12)
# # Show the plot
# plt.tight_layout()
# plt.show()

In [10]:

# remaining_ratio = [0.1, 0.2, 0.3, 0.4]
# robust_acc_constant = [45.7, 47.28, 46.82, 45.72]
# robust_acc_kaiming_uniform = [44.67, 45.59, 45.04, 45.23]
# robust_acc_xavier_normal = [43.65, 44.61, 44.96, 44.64]

# # Create the plot with different initialization methods
# fig, ax = plt.subplots(figsize=(10, 6))

# # Plot Robust Accuracy for different initialization methods
# ax.plot(remaining_ratio, robust_acc_constant, marker='s', linestyle='-', color='red', label='Signed Kaiming Constant')
# ax.plot(remaining_ratio, robust_acc_kaiming_uniform, marker='o', linestyle='-', color='orange', label='Kaiming Uniform')
# ax.plot(remaining_ratio, robust_acc_xavier_normal, marker='^', linestyle='-', color='green', label='Xavier Normal')
# ax.axhline(y=51.00, color='purple', linestyle='--', linewidth=2)

# ax.set_title("(e) Resnet @ CIFAR-10", fontsize=16, fontweight='bold')
# ax.set_xlabel("Remaining Ratio", fontsize=14, fontweight='bold')
# ax.set_ylabel("Robust Accuracy (%)", fontsize=14, fontweight='bold')
# ax.set_xlim(-0.05, 1)
# ax.set_ylim(20, 55)
# ax.grid(True)
# ax.legend(fontsize=12)
# ax.set_yticks(np.arange(33, 51, 3))
# ax.set_xticks(np.arange(0, 1, 0.1))

# ax.tick_params(axis='both', which='major', labelsize=12)